In [24]:
from IPython.display import Audio, display
import plotly.express as px
import numpy as np
from tqdm import tqdm

In [2]:
import sys
import os
from jiwer import wer, cer

import glob
import torch
import pandas as pd
import matplotlib.pyplot as pltm

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [40]:
df = pd.read_csv("../workdir/csvs/submission.csv")
all_vocabs = {}
vocabs_idx = {}
with open("../workdir/csvs/vocab.txt") as f:
    for idx,l in enumerate(f.readlines()):
        all_vocabs[l.strip()] = idx
        vocabs_idx[idx]  = l.strip()

In [10]:
metadata = pd.read_csv("/app/dataset/metadata/train_metadata.csv")
valid = pd.read_csv("/app/dataset/valid_data.csv")
train = pd.read_csv("/app/dataset/train_data_with_openasr.csv")
valid_subset = pd.read_csv("/app/dataset/valid_data_subset.csv")
ood_data = pd.read_csv("/app/dataset/metadata/annoated.csv",delimiter="	")

/tmp/ipykernel_2726510/948830026.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("/app/dataset/train_data_with_openasr.csv")


In [14]:
VOCAB = set(['!', '?', ',', '।', '-', '‘', '’', '"', ';', '–', "'", ':', '/', '.', '“', '”'])

In [25]:
hit = 0
total = 0 
not_vocab = []
for sent in tqdm(train.sentence.apply(lambda x: "".join([t for t in x if t not in VOCAB]))):
    for w in sent.split(" "):
        if w in all_vocabs:
            hit+=1
        else:
            not_vocab.append(w)
        total+=1

100%|██████████| 989602/989602 [00:02<00:00, 344635.98it/s]


In [82]:
import editdistance

In [102]:
def cer(word1, word2):
    """
    Calculate the Character Error Rate (CER) between two words.
    """
    # Calculate Levenshtein distance
    distance = editdistance.eval(word1, word2)
    
    # Calculate CER
    return distance / (len(word2)+1e-5)

def fix_sentence(sentence, vocabulary,vocabulary_idx, T=0.05):
    """
    Fix the sentence using the given vocabulary and threshold T.
    """
    # Split the sentence into words
    words = sentence.split()
    
    # For each word in the sentence
    for i, word in enumerate(words):
        # If the word is not in the vocabulary
        if word not in vocabulary:
            # Calculate IOU with each word in the vocabulary
            ious = [cer(word, vocab_word) for vocab_word in vocabulary]
            
            # Get the maximum IOU and its index
            max_iou = min(ious)
            max_iou_index = ious.index(max_iou)
            
            # If the maximum IOU is above the threshold T
            if max_iou < T:
                print(f"Replacing {words[i]} with {vocabulary_idx[max_iou_index]}")
                # Replace the word in the sentence with the word from the vocabulary
                words[i] = vocabulary_idx[max_iou_index]
    # Join the words back into a sentence
    return ' '.join(words)

In [103]:
fixed_sebntence = []
for sent in tqdm(df.sentence): 
    fixed_sebntence.append(fix_sentence(sent,all_vocabs,vocabs_idx))

100%|██████████| 1000/1000 [06:19<00:00,  2.63it/s]


In [104]:
df["fixed"] = fixed_sebntence

In [106]:
np.mean([wer(a,b) for a,b in zip(df.gts,df.sentence)])

0.24966049700213785

In [107]:
df

,id,sentence,gts,fixed
0,765938ad67d9,তিনি এই কর্মসূচির গুরুত্ব জাতীয় নিরাপত্তা এবং ...,"তিনি এই কর্মসূচির গুরুত্ব জাতীয় নিরাপত্তা, এব...",তিনি এই কর্মসূচির গুরুত্ব জাতীয় নিরাপত্তা এবং ...
1,60ff1ac8da38,সারদা দেবী সমাজসেবায় সক্রিয় ছিলেন।,সারদা দেবী সমাজ সেবায় সক্রিয় ছিলেন।,সারদা দেবী সমাজসেবায় সক্রিয় ছিলেন।
2,6d554cd13741,বিবেকানন্দ উত্তরে বলেছিলেন ভারতকে ভালোবেসে।,"বিবেকানন্দ উত্তরে বলেছিলেন, ""ভারতকে ভালবেসে""।",বিবেকানন্দ উত্তরে বলেছিলেন ভারতকে ভালোবেসে।
3,3a023659f5bf,এই ক্রম অনুসারে পরবর্তী লাইনগুলি চলতে থাকে।,এই ক্রম অনুসারে পরবর্তী লাইন গুলো চলতে থাকে।,এই ক্রম অনুসারে পরবর্তী লাইনগুলি চলতে থাকে।
4,6faacc0aceaa,বাহাদুর বাদ ঘাট স্টেশন বন্ধ হওয়ার পর নারায়ণগঞ্...,বাহাদুরাবাদ ঘাট স্টেশন বন্ধ হওয়ার পর নারায়ণগ...,বাহাদুর বাদ ঘাট স্টেশন বন্ধ হওয়ার পর নারায়ণগঞ্...
...,...,...,...,...
995,f0abd8f6971a,এর প্রশান্ত মহাসাগরের তীরে রয়েছে সরু সম্ভমে এল...,এর প্রশান্ত মহাসাগরের তীরে রয়েছে সরু সমভূমি এ...,এর প্রশান্ত মহাসাগরের তীরে রয়েছে সরু সম্ভমে এল...
996,2309556fca54,পাওয়ার চেয়ার ব্যবহারকারীদের জন্যে এটিই একমাত্র...,পাওয়ার চেয়ার ব্যবহারকারীদের জন্যে এটিই একমাত...,পাওয়ার চেয়ার ব্যবহারকারীদের জন্যে এটিই একমাত্র...
997,8d512bd29b64,তিনি ইস্তাম্বুলে মারা যান।,তিনি ইস্তাম্বুলে মারা যান।,তিনি ইস্তাম্বুলে মারা যান।
998,fffd69cdd76c,আবদুল লতিফ আঞ্জুমানের সঙ্গে প্রত্যক্ষ ভাবে জড়ি...,আবদুল লতিফ আঞ্জুমানের সঙ্গে প্রত্যক্ষভাবে জড়ি...,আবদুল লতিফ আঞ্জুমানের সঙ্গে প্রত্যক্ষ ভাবে জড়ি...
